# Lesson 2: Empowering Agents with Tools (Function Calling)

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:

1. **Understand** what tools are in the context of AI agents
2. **Create** Python functions that agents can use as tools
3. **Implement** proper type hints and docstrings for automatic tool schema generation
4. **Observe** how agents decide when and how to use tools
5. **Debug** tool execution to understand the agent's decision-making process
6. **Design** effective tools for real-world IT support scenarios

## 📚 Quick Recap: Lesson 1

In the previous lesson, you learned:
- ✅ How to create a basic `LlmAgent` with personality and instructions
- ✅ How to use `Runner` and `InMemorySessionService` for conversations
- ✅ How to maintain context across multiple conversation turns

**Limitation**: Our agents could only **chat** - they couldn't take actions or access information.

## 🚀 What's New in This Lesson?

We're going to give our IT support agent **superpowers** by adding **tools** (also known as function calling). With tools, agents can:
- 🎫 Look up ticket statuses
- 📚 Search knowledge bases
- 🔄 Restart services
- 🖥️ Check system statuses
- And much more!

## 🏢 Use Case: Enhanced IT Support

We'll enhance our IT Support agent from Lesson 1 by giving it access to real (well, mock) IT support tools. This transforms it from a conversational assistant into an **action-taking agent**.

---

## 🔧 Part 1: Environment Setup

Let's install packages and set up our environment (same as Lesson 1).

In [ ]:
# Install the Google Agent Development Kit and dependencies
!pip install -q google-adk litellm openai python-dotenv nest-asyncio

print("✅ Packages installed successfully!")

In [ ]:
# Core ADK imports
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.models.lite_llm import LiteLlm  # For using OpenAI and other LLM providers
from google.genai import types

# System imports
import os
import asyncio
from typing import Dict, List, Optional
from datetime import datetime, timedelta
import random

print("✅ Imports successful!")

In [ ]:
# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

# Configure which OpenAI model to use
# Options: "gpt-4o", "gpt-4o-mini", "gpt-4-turbo", "gpt-3.5-turbo", etc.
OPENAI_MODEL = "gpt-4o-mini"  # Using gpt-4o-mini for cost efficiency
print(f"🤖 Selected Model: {OPENAI_MODEL}")

---

## 🛠️ Part 2: What Are Tools in ADK?

### The Problem Without Tools

In Lesson 1, our agent could only generate text responses. It couldn't:
- ❌ Look up real data
- ❌ Perform actions
- ❌ Access external systems
- ❌ Execute code

### The Solution: Function Calling (Tools)

**Tools** are Python functions that agents can call to extend their capabilities. Think of them as giving your agent hands to interact with the world!

### How Tools Work in ADK

```
1. User asks: "What's the status of ticket #1234?"
   ↓
2. Agent analyzes: "I need to use the check_ticket_status tool"
   ↓
3. ADK calls: check_ticket_status(ticket_id="1234")
   ↓
4. Function returns: {"status": "in_progress", "assigned_to": "tech_jane"}
   ↓
5. Agent responds: "Ticket #1234 is in progress, assigned to Jane."
```

### The Magic: Automatic Schema Generation

ADK automatically inspects your Python functions and generates a schema that tells the LLM:
- **Function name**: What the tool is called
- **Docstring**: What the tool does
- **Type hints**: What parameters it expects
- **Return type**: What it returns

This means you just write **normal Python functions** with good documentation!

### Anatomy of a Good Tool Function

```python
def tool_name(param1: str, param2: int = 10) -> Dict[str, any]:
    """
    Clear description of what this tool does.
    
    Args:
        param1 (str): Description of param1 (required)
        param2 (int): Description of param2 (optional, default=10)
    
    Returns:
        Dict: Description of return value
    """
    # Your implementation
    return {"result": "value"}
```

**Key points:**
1. ✅ **Type hints** on all parameters
2. ✅ **Comprehensive docstring** explaining purpose
3. ✅ **Clear parameter descriptions** in Args section
4. ✅ **Return dictionary** with descriptive keys
5. ✅ **Meaningful function name** (describes the action)

---

## 📊 Part 3: Creating Mock IT Support Tools

Let's create realistic mock data and tools for our IT support system.

### 3.1: Mock Database Setup

First, we'll create in-memory databases to simulate real systems.

In [ ]:
# Mock Ticket Database
TICKETS_DB = {
    "T-1001": {
        "id": "T-1001",
        "status": "open",
        "priority": "high",
        "subject": "Cannot access email",
        "description": "User reports unable to login to university email system",
        "assigned_to": "tech_jane",
        "created_at": "2025-10-05 09:30:00",
        "updated_at": "2025-10-05 14:20:00"
    },
    "T-1002": {
        "id": "T-1002",
        "status": "in_progress",
        "priority": "medium",
        "subject": "Printer not working",
        "description": "Library printer 2B not responding",
        "assigned_to": "tech_mike",
        "created_at": "2025-10-04 11:15:00",
        "updated_at": "2025-10-06 08:00:00"
    },
    "T-1003": {
        "id": "T-1003",
        "status": "resolved",
        "priority": "low",
        "subject": "Wi-Fi connection issues",
        "description": "Intermittent connection drops in Building A",
        "assigned_to": "tech_sarah",
        "created_at": "2025-10-03 14:00:00",
        "updated_at": "2025-10-05 16:30:00",
        "resolved_at": "2025-10-05 16:30:00"
    },
    "T-1004": {
        "id": "T-1004",
        "status": "open",
        "priority": "critical",
        "subject": "Campus portal down",
        "description": "Students cannot access course materials",
        "assigned_to": None,
        "created_at": "2025-10-06 07:00:00",
        "updated_at": "2025-10-06 07:00:00"
    }
}

# Mock Knowledge Base
KNOWLEDGE_BASE = [
    {
        "id": "KB-001",
        "title": "How to Reset Your Password",
        "content": "To reset your password: 1) Go to portal.university.edu 2) Click 'Forgot Password' 3) Enter your student ID 4) Check your email for reset link 5) Create a new password (min 8 characters, include numbers and symbols)",
        "tags": ["password", "login", "account", "reset"]
    },
    {
        "id": "KB-002",
        "title": "Connecting to University Wi-Fi",
        "content": "To connect to eduroam: 1) Select 'eduroam' network 2) Username: studentid@university.edu 3) Password: your portal password 4) Accept security certificate 5) If issues persist, forget the network and reconnect",
        "tags": ["wifi", "network", "eduroam", "connection"]
    },
    {
        "id": "KB-003",
        "title": "Printing from Library Computers",
        "content": "To print: 1) Send document to 'University-Print' queue 2) Go to any library printer 3) Tap your student ID card 4) Select your document 5) Choose print options 6) Print credit will be deducted automatically",
        "tags": ["printing", "library", "printer", "documents"]
    },
    {
        "id": "KB-004",
        "title": "Email Not Working - Common Solutions",
        "content": "Email issues? Try: 1) Clear browser cache and cookies 2) Try different browser 3) Check if caps lock is on 4) Verify username format (firstname.lastname@university.edu) 5) Reset password if needed 6) Check if account is active",
        "tags": ["email", "outlook", "gmail", "login", "troubleshooting"]
    },
    {
        "id": "KB-005",
        "title": "VPN Setup for Off-Campus Access",
        "content": "Setting up VPN: 1) Download Cisco AnyConnect from IT portal 2) Install the application 3) Server address: vpn.university.edu 4) Login with university credentials 5) Accept connection 6) You can now access library resources",
        "tags": ["vpn", "remote", "access", "cisco", "anyconnect"]
    }
]

# Mock System Status
SYSTEMS_STATUS = {
    "email": {"status": "operational", "uptime": "99.9%", "last_incident": "2025-09-28"},
    "portal": {"status": "degraded", "uptime": "95.2%", "last_incident": "2025-10-06"},
    "wifi": {"status": "operational", "uptime": "99.5%", "last_incident": "2025-10-01"},
    "printing": {"status": "operational", "uptime": "98.8%", "last_incident": "2025-10-03"},
    "vpn": {"status": "operational", "uptime": "99.7%", "last_incident": "2025-09-15"},
    "library": {"status": "operational", "uptime": "99.9%", "last_incident": "2025-09-20"}
}

# Mock Services that can be restarted
SERVICES = ["email", "portal", "wifi", "printing", "vpn", "library"]

print("✅ Mock databases created!")
print(f"📊 Loaded {len(TICKETS_DB)} tickets")
print(f"📚 Loaded {len(KNOWLEDGE_BASE)} knowledge base articles")
print(f"🖥️ Monitoring {len(SYSTEMS_STATUS)} systems")

### 3.2: Creating Tool Functions

Now let's create the actual tool functions. Notice how we use:
- **Type hints** for all parameters
- **Detailed docstrings** explaining what the tool does
- **Print statements** to show when tools are called (for debugging)
- **Dictionary returns** with descriptive keys

#### ⚠️ Important: Google AI Studio vs Vertex AI

**Google AI Studio** (which we're using) has a limitation: **it doesn't support default parameter values** in function tools. This means:

❌ **Won't work with Google AI Studio:**
```python
def search_knowledge_base(query: str, max_results: int = 3):
```

✅ **Works with Google AI Studio:**
```python
def search_knowledge_base(query: str):
    max_results = 3  # Set default inside function
```

If you switch to **Vertex AI** in the future, you can use default parameters. For now, we'll use fixed values inside our functions.

In [ ]:
def check_ticket_status(ticket_id: str) -> Dict[str, any]:
    """
    Retrieves the current status and details of a support ticket.
    
    Use this tool when a user asks about the status of their support ticket.
    The ticket ID is usually in the format 'T-XXXX'.
    
    Args:
        ticket_id (str): The unique ticket identifier (e.g., 'T-1001')
    
    Returns:
        Dict: Ticket information including status, priority, subject, and assigned technician
    """
    print(f"🔧 [TOOL CALLED] check_ticket_status(ticket_id='{ticket_id}')")
    
    # Normalize ticket ID (handle case variations)
    ticket_id = ticket_id.upper().strip()
    
    if ticket_id in TICKETS_DB:
        ticket = TICKETS_DB[ticket_id]
        return {
            "success": True,
            "ticket_id": ticket["id"],
            "status": ticket["status"],
            "priority": ticket["priority"],
            "subject": ticket["subject"],
            "assigned_to": ticket["assigned_to"] if ticket["assigned_to"] else "Unassigned",
            "created_at": ticket["created_at"],
            "last_updated": ticket["updated_at"]
        }
    else:
        return {
            "success": False,
            "error": f"Ticket {ticket_id} not found in the system",
            "suggestion": "Please verify the ticket ID format (e.g., T-1001)"
        }


def search_knowledge_base(query: str) -> Dict[str, any]:
    """
    Searches the IT support knowledge base for relevant articles.
    
    Use this tool to find help articles, guides, and troubleshooting steps
    for common IT issues. The search looks through article titles, content, and tags.
    Returns up to 3 most relevant articles.
    
    Args:
        query (str): The search query (e.g., 'password reset', 'wifi connection')
    
    Returns:
        Dict: Search results with relevant knowledge base articles
    """
    print(f"🔧 [TOOL CALLED] search_knowledge_base(query='{query}')")
    
    query_lower = query.lower()
    results = []
    max_results = 3  # Fixed value since Google AI doesn't support default parameters
    
    # Simple search: look for query terms in title, content, and tags
    for article in KNOWLEDGE_BASE:
        relevance_score = 0
        
        # Check title
        if query_lower in article["title"].lower():
            relevance_score += 3
        
        # Check tags
        for tag in article["tags"]:
            if query_lower in tag.lower() or tag.lower() in query_lower:
                relevance_score += 2
        
        # Check content
        if query_lower in article["content"].lower():
            relevance_score += 1
        
        if relevance_score > 0:
            results.append({
                "id": article["id"],
                "title": article["title"],
                "content": article["content"],
                "relevance": relevance_score
            })
    
    # Sort by relevance and limit results
    results.sort(key=lambda x: x["relevance"], reverse=True)
    results = results[:max_results]
    
    if results:
        return {
            "success": True,
            "query": query,
            "results_count": len(results),
            "articles": results
        }
    else:
        return {
            "success": False,
            "query": query,
            "message": "No relevant articles found",
            "suggestion": "Try rephrasing your query or contact support directly"
        }


def check_system_status(system: str) -> Dict[str, any]:
    """
    Checks the operational status of a university IT system.
    
    Use this tool to verify if a system is operational, degraded, or down.
    Available systems: email, portal, wifi, printing, vpn, library
    
    Args:
        system (str): The system name to check (e.g., 'email', 'wifi', 'portal')
    
    Returns:
        Dict: System status information including uptime and last incident
    """
    print(f"🔧 [TOOL CALLED] check_system_status(system='{system}')")
    
    system_lower = system.lower().strip()
    
    if system_lower in SYSTEMS_STATUS:
        status_info = SYSTEMS_STATUS[system_lower]
        return {
            "success": True,
            "system": system_lower,
            "status": status_info["status"],
            "uptime": status_info["uptime"],
            "last_incident": status_info["last_incident"],
            "message": f"System '{system_lower}' is currently {status_info['status']}"
        }
    else:
        return {
            "success": False,
            "error": f"System '{system}' not found",
            "available_systems": list(SYSTEMS_STATUS.keys())
        }


def restart_service(service_name: str) -> Dict[str, any]:
    """
    Attempts to restart a university IT service.
    
    Use this tool when troubleshooting requires restarting a service.
    This is a mock function that simulates a service restart.
    In production, this would trigger actual service management systems.
    The service will be restarted normally (not forced).
    
    Args:
        service_name (str): The service to restart (e.g., 'email', 'portal')
    
    Returns:
        Dict: Result of the restart operation
    """
    print(f"🔧 [TOOL CALLED] restart_service(service_name='{service_name}')")
    
    service_lower = service_name.lower().strip()
    
    if service_lower not in SERVICES:
        return {
            "success": False,
            "error": f"Service '{service_name}' not found",
            "available_services": SERVICES
        }
    
    # Simulate restart with random success (90% success rate)
    restart_successful = random.random() < 0.9
    
    if restart_successful:
        # Update system status to operational if it was degraded
        if service_lower in SYSTEMS_STATUS:
            SYSTEMS_STATUS[service_lower]["status"] = "operational"
        
        return {
            "success": True,
            "service": service_lower,
            "message": f"Service '{service_lower}' has been successfully restarted",
            "status": "operational"
        }
    else:
        return {
            "success": False,
            "service": service_lower,
            "error": "Restart failed - service did not respond",
            "suggestion": "Escalate to senior technician or contact infrastructure team"
        }

print("✅ Tool functions created!")
print("\n📋 Available tools:")
print("  1. check_ticket_status(ticket_id) - Look up support ticket information")
print("  2. search_knowledge_base(query) - Search help articles (returns top 3)")
print("  3. check_system_status(system) - Check if systems are operational")
print("  4. restart_service(service_name) - Restart IT services")
print("\n⚠️  Note: These tools are optimized for Google AI Studio (no default parameters)")

---

## 🤖 Part 4: Creating an Agent with Tools

Now let's create an enhanced IT support agent that can use these tools!

### Key Differences from Lesson 1:
1. We add a `tools` parameter with our function list
2. We update the instructions to tell the agent about its new capabilities
3. The agent will automatically decide when to use tools based on user requests

In [ ]:
# Create session service and app configuration
session_service = InMemorySessionService()
APP_NAME = "it_support_tools_app"

# Create an IT Support Agent WITH TOOLS using OpenAI
it_support_agent_with_tools = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),  # Use OpenAI model via LiteLLM
    name="it_support_with_tools",
    tools=[
        check_ticket_status,
        search_knowledge_base,
        check_system_status,
        restart_service
    ],
    instruction="""
    You are Alex, an enhanced IT Support Assistant for TechHelp Solutions.
    
    You now have access to powerful tools that let you take actions:
    
    AVAILABLE TOOLS:
    1. check_ticket_status - Look up any support ticket by ID
    2. search_knowledge_base - Find help articles and guides
    3. check_system_status - Verify if university systems are operational
    4. restart_service - Restart services when troubleshooting
    
    WHEN TO USE TOOLS:
    - If a user mentions a ticket number (like T-1001), use check_ticket_status
    - If a user asks how to do something, search the knowledge base first
    - If a user reports a system issue, check the system status
    - Only restart services when troubleshooting steps have been exhausted
    
    BEST PRACTICES:
    - Always use tools when you have them available for a task
    - Explain what tool you're using and why
    - If a tool returns an error, explain it clearly to the user
    - Provide the information from tools in a user-friendly way
    - After using tools, give actionable next steps
    
    YOUR PERSONALITY:
    - Friendly and helpful
    - Professional but approachable
    - Proactive in using tools to help users
    - Clear about what actions you're taking
    
    Remember: You're not just chatting anymore - you can actually DO things to help users!
    """
)

# Create the runner
runner = Runner(
    app_name=APP_NAME,
    agent=it_support_agent_with_tools,
    session_service=session_service
)

print("✅ IT Support Agent with Tools created!")
print(f"🔧 Agent has access to {len(it_support_agent_with_tools.tools)} tools")
print("📋 Agent Name:", it_support_agent_with_tools.name)
print(f"🤖 Model: OpenAI {OPENAI_MODEL}")

### Create Helper Function

Same async helper function from Lesson 1, with added support for tool visibility.

In [ ]:
# Track created sessions
_created_sessions = set()

async def chat_with_agent_async(user_message: str, session_id: str = "session_001", user_id: str = "student_001", show_all_events: bool = False):
    """
    Send a message to the IT support agent and print the response (async version).
    """
    # Create session if needed
    session_key = (session_id, user_id)
    if session_key not in _created_sessions:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=user_id,
            session_id=session_id,
            state={}
        )
        _created_sessions.add(session_key)
    
    content = types.Content(role='user', parts=[types.Part(text=user_message)])
    events = runner.run_async(user_id=user_id, session_id=session_id, new_message=content)
    
    print(f"\n{'='*70}")
    print(f"👤 You: {user_message}")
    print(f"{'='*70}\n")
    
    tool_calls = []
    final_response = None
    
    async for event in events:
        if show_all_events:
            print(f"[DEBUG EVENT]: {event}\n")
        
        if event.is_final_response():
            final_response = event.content.parts[0].text
    
    if final_response:
        print(f"🤖 Alex: {final_response}")
        print(f"\n{'='*70}\n")
        return final_response
    else:
        print("⚠️ No response received from agent.")
        return None

def chat_with_agent(user_message: str, session_id: str = "session_001", user_id: str = "student_001", show_all_events: bool = False):
    """
    Synchronous wrapper for chat_with_agent_async.
    """
    try:
        loop = asyncio.get_running_loop()
        import nest_asyncio
        nest_asyncio.apply()
        return asyncio.run(chat_with_agent_async(user_message, session_id, user_id, show_all_events))
    except RuntimeError:
        return asyncio.run(chat_with_agent_async(user_message, session_id, user_id, show_all_events))

print("✅ Helper function created!")

---

## 🎬 Part 5: Demonstration - Agent Using Tools

Let's see our enhanced agent in action! Watch for the `[TOOL CALLED]` messages that show when the agent uses tools.

### Demo 1: Checking Ticket Status

In [ ]:
chat_with_agent(
    "Hi! Can you check the status of my ticket T-1002?",
    session_id="demo_session_1"
)

### Demo 2: Searching Knowledge Base

In [ ]:
chat_with_agent(
    "I forgot my password. How do I reset it?",
    session_id="demo_session_2"
)

### Demo 3: Checking System Status

In [ ]:
chat_with_agent(
    "I can't access the university portal. Is it down?",
    session_id="demo_session_3"
)

### Demo 4: Multiple Tools in One Conversation

In [ ]:
chat_with_agent(
    "Can you check ticket T-1001 and also tell me how to connect to Wi-Fi?",
    session_id="demo_session_4"
)

### Demo 5: Service Restart (Advanced)

In [ ]:
chat_with_agent(
    "The portal is running really slow. Can you try restarting it?",
    session_id="demo_session_5"
)

### Your Turn - Try It!

Experiment with different requests to see how the agent uses tools:

In [ ]:
# Try your own query!
your_query = "What's the status of ticket T-1004 and is the email system working?"  # Modify this

chat_with_agent(your_query, session_id="my_test_session")

---

## 🔍 Part 6: Understanding Tool Execution Flow

### What Just Happened?

When you send a message to the agent:

1. **Agent analyzes** the user's request
2. **Agent decides** which tool(s) to use (if any)
3. **ADK executes** the Python function(s)
4. **Tool returns** data to the agent
5. **Agent synthesizes** a natural language response

### How Does the Agent Know When to Use Tools?

The LLM (Gemini) reads:
- ✅ **Function name**: `check_ticket_status` → "This checks tickets"
- ✅ **Docstring**: Detailed explanation of what the tool does
- ✅ **Parameters**: What information it needs
- ✅ **Type hints**: What format the parameters should be

Then it **intelligently decides** if and when to call the tool!

### Why Do We See `[TOOL CALLED]` Messages?

We added `print()` statements in our tool functions for **debugging**. This helps us:
- See exactly when tools are called
- Verify the agent is using tools correctly
- Understand the agent's decision-making process

**In production**, you'd remove these or log them properly.

### Tool Execution is Automatic

Notice you **never explicitly tell** the agent "use tool X". The agent:
- Interprets user intent
- Selects appropriate tools
- Extracts parameters from context
- Calls tools automatically

This is the power of **function calling** in modern LLMs!

---

## 🎓 Part 7: Student Exercises

Now it's your turn to create custom tools!

### Exercise 1: Create a `list_all_tickets` Tool (Beginner)

**Task**: Create a tool that lists all open tickets.

**Requirements**:
1. Function name: `list_all_tickets`
2. Optional parameter: `status` (filter by status: "open", "in_progress", "resolved")
3. Returns a dictionary with list of tickets
4. Include proper docstring and type hints

**Hint**: Loop through `TICKETS_DB` and filter by status if provided.

In [ ]:
# Exercise 1: Your code here

def list_all_tickets(status: str) -> Dict[str, any]:
    """
    Lists all support tickets, optionally filtered by status.
    
    Use this tool to get an overview of all tickets in the system.
    If status is "all", returns all tickets. Otherwise filters by the specified status.
    
    Args:
        status (str): Filter by status ("open", "in_progress", "resolved", or "all" for no filter)
    
    Returns:
        Dict: List of tickets matching the criteria
    """
    print(f"🔧 [TOOL CALLED] list_all_tickets(status='{status}')")
    
    # TODO: Implement the logic to list tickets
    # If status is "all", return all tickets
    # Otherwise, filter by that status
    # Hint: Loop through TICKETS_DB.values() and check each ticket's status
    
    pass  # Replace this with your implementation

# Test your tool directly
# print(list_all_tickets(status="all"))
# print(list_all_tickets(status="open"))

### Exercise 2: Create a `get_technician_workload` Tool (Intermediate)

**Task**: Create a tool that shows how many tickets each technician is handling.

**Requirements**:
1. Function name: `get_technician_workload`
2. Optional parameter: `technician_name` (show specific technician or all)
3. Count open and in_progress tickets only
4. Return dictionary with technician names and ticket counts

**Challenge**: Also show which tickets are assigned to each technician.

In [ ]:
# Exercise 2: Your code here

def get_technician_workload(technician_name: str) -> Dict[str, any]:
    """
    Shows how many tickets a technician is handling.
    
    Use "all" to see workload for all technicians, or specify a specific
    technician name (e.g., "tech_jane") to see their workload.
    Only counts open and in_progress tickets.
    
    Args:
        technician_name (str): The technician to check ("all" for all technicians, or specific name like "tech_jane")
    
    Returns:
        Dict: Technician workload information with ticket counts
    """
    print(f"🔧 [TOOL CALLED] get_technician_workload(technician_name='{technician_name}')")
    
    # TODO: Implement logic to:
    # 1. Loop through TICKETS_DB.values()
    # 2. Count tickets per technician (only open/in_progress)
    # 3. If technician_name is "all", show all technicians
    # 4. If technician_name is specific, filter to that technician
    # 5. Return structured data with counts and ticket IDs
    
    pass  # Replace with your implementation

# Test your tool
# print(get_technician_workload(technician_name="all"))
# print(get_technician_workload(technician_name="tech_jane"))

### Exercise 3: Add Your Tools to the Agent (Required)

**Task**: Create a new agent that includes your custom tools.

**Instructions**:
1. Copy the agent creation code
2. Add your new tools to the `tools` list
3. Update the instructions to mention the new tools
4. Test it with queries that would use your tools

In [ ]:
# Exercise 3: Create enhanced agent with your custom tools

# TODO: Create a new agent that includes all original tools PLUS your custom tools
# enhanced_agent = LlmAgent(
#     model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
#     name="enhanced_it_support",
#     tools=[
#         check_ticket_status,
#         search_knowledge_base,
#         check_system_status,
#         restart_service,
#         # Add your custom tools here
#     ],
#     instruction="""
#     # TODO: Update instructions to mention new tools
#     """
# )

# TODO: Create a new runner with your enhanced agent

# TODO: Test with queries like:
# "Show me all open tickets"
# "How many tickets does tech_jane have?"

### Bonus Exercise: Create a `create_ticket` Tool (Advanced)

**Task**: Create a tool that can create new support tickets.

**Requirements**:
1. Parameters: `subject`, `description`, `priority`
2. Generate a new ticket ID (e.g., T-1005, T-1006, etc.)
3. Add the ticket to `TICKETS_DB`
4. Set initial status as "open", assigned_to as None
5. Return the new ticket information

**Challenge**: Make the agent able to create tickets when users report issues!

In [ ]:
# Bonus Exercise: Your code here

def create_ticket(subject: str, description: str, priority: str) -> Dict[str, any]:
    """
    Creates a new support ticket in the system.
    
    Use this tool when a user reports an issue that needs to be tracked.
    The ticket will be created with status "open" and no assigned technician.
    
    Args:
        subject (str): Brief description of the issue
        description (str): Detailed description of the problem
        priority (str): Priority level ("low", "medium", "high", or "critical")
    
    Returns:
        Dict: Information about the newly created ticket
    """
    print(f"🔧 [TOOL CALLED] create_ticket(subject='{subject}', description='{description}', priority='{priority}')")
    
    # TODO: Implement ticket creation logic
    # 1. Generate new ticket ID (find highest number in TICKETS_DB and add 1)
    # 2. Create ticket dict with all required fields
    # 3. Set status="open", assigned_to=None
    # 4. Add timestamps
    # 5. Add to TICKETS_DB
    # 6. Return ticket information
    
    pass

---

## 🎯 Part 8: Key Takeaways

Congratulations! You've learned how to empower AI agents with tools!

### What You Learned Today ✅

1. **Tool Concept**
   - Tools are Python functions that extend agent capabilities
   - Agents automatically decide when to use tools
   - Function calling bridges conversation and action

2. **Creating Effective Tools**
   - Use comprehensive docstrings
   - Add type hints for all parameters
   - Return dictionaries with descriptive keys
   - Choose meaningful function names
   - Design for the LLM to understand

3. **ADK's Automatic Magic**
   - ADK inspects function signatures automatically
   - Converts Python functions to LLM-understandable schemas
   - Handles parameter validation and type conversion
   - Manages tool execution flow

4. **Best Practices**
   - Keep tools focused (single responsibility)
   - Handle errors gracefully
   - Return structured data
   - Use optional parameters with defaults
   - Add debug logging for development

### Design Patterns Learned 🎨

1. **Mock Data Pattern**: Create realistic test data without external dependencies
2. **Tool Debugging**: Use print statements to observe agent behavior
3. **Error Handling**: Return success/error information in structured format
4. **Optional Parameters**: Use defaults to make tools flexible

### What's Coming Next 🚀

In the upcoming lessons, you'll learn:

- **Lesson 3: Multi-Agent Systems**
  - Create specialized agents that work together
  - Build agent teams (triage → specialist → resolution)
  - Implement agent-to-agent communication
  - Each agent has different tools and expertise

- **Lesson 4: Advanced State Management**
  - Use persistent storage (Firestore)
  - Implement custom state variables
  - Build long-term memory systems
  - Track conversation context across sessions

- **Lesson 5: Integration with MCP**
  - Connect ADK agents to MCP servers
  - Use real external tools and APIs
  - Build production-ready systems
  - Combine ADK and MCP for maximum power

### Real-World Applications 🌍

The skills you learned today apply to:
- **Customer support**: Automated ticket management
- **IT operations**: System monitoring and troubleshooting
- **Data analysis**: Querying databases and generating reports
- **Workflow automation**: Executing business processes
- **Personal assistants**: Calendar, email, task management

### Important Reminders ⚠️

1. **Mock vs. Real**: Today we used mock functions. In production:
   - Connect to real databases
   - Use actual APIs
   - Implement proper authentication
   - Add comprehensive error handling

2. **Security**: When creating real tools:
   - Validate all inputs
   - Implement access controls
   - Log all tool executions
   - Rate limit expensive operations

3. **Testing**: Always test tools:
   - Independently (unit tests)
   - With the agent (integration tests)
   - Edge cases and error conditions

### Resources 📚

- [ADK Function Tools Documentation](https://google.github.io/adk-docs/tools/function-tools/)
- [ADK Tools Overview](https://google.github.io/adk-docs/tools/)
- [Google Codelabs - ADK with Tools](https://codelabs.developers.google.com/)
- [ADK GitHub Examples](https://github.com/google/adk-python)

---

### 🎓 Next Steps

1. ✅ Complete all exercises above
2. ✅ Create at least 2 custom tools
3. ✅ Test your tools with various queries
4. ✅ Experiment with tool combinations
5. ✅ Think about tools for your own use cases
6. 🚀 Get ready for Lesson 3: Multi-Agent Systems!

**Great work! You've unlocked the power of function calling! 🎉**

---